# Нейронная сеть Хопфилда

Импортируем инструменты

In [12]:
import numpy as np

In [29]:
y = np.array(np.sign(np.random.random(size=(3, 8)) - 0.5), dtype=np.int32)
print(y)

[[-1  1  1  1  1  1 -1  1]
 [-1  1 -1 -1  1 -1  1  1]
 [-1  1  1 -1 -1 -1  1 -1]]


In [30]:
hn = HopfieldNetwork().fit(y)

In [31]:
hn.predict(y)

array([[ 1,  1,  1,  1,  1,  1, -1,  1],
       [-1, -1, -1,  1,  1, -1,  1, -1],
       [-1,  1,  1, -1, -1, -1,  1, -1]])